In [30]:
ANLI_R3_train = pd.read_json('/Users/ardsnijders/Documents/GitHub/Lisa/active_learning/resources/ard_data/anli_v1.0/R3/train.jsonl', lines=True)

ANLI_R3_train[ANLI_R3_train['uid']=='0e98e1de-3c7f-4906-bed1-76d507555068']

,uid,context,hypothesis,label,model_label,emturk,genre,reason,tag
81915,0e98e1de-3c7f-4906-bed1-76d507555068,4 of 9 Courtesy of Heidi Klum Diet Coke Campai...,Klum wore multiple looks for the campaign.,e,e,False,news,,r3_train


In [26]:
path = '/Users/ardsnijders/Documents/GitHub/Lisa/active_learning/results/ard_output_dir_results/acquisition_IDs/array/OOD_MNLI_active_learning_array_donderdag/roberta-large'

strategies = ['bald', 'dal', 'max-entropy', 'mc-max-entropy', 'random']

df = pd.read_csv('/Users/ardsnijders/Documents/GitHub/Lisa/active_learning/results/ard_output_dir_results/acquisition_IDs/array/OOD_MNLI_active_learning_array_donderdag/roberta-large/bald/42/acquisition_ids.csv')

df

,round,ID,Dataset,Label
0,0,0e98e1de-3c7f-4906-bed1-76d507555068,ANLI,entailment
1,0,114990,WANLI,contradiction
2,0,b910d7cd-76bb-40b5-81bb-b9e4a9f53e81,ANLI,entailment
3,0,fc5aa613-243c-4d22-afe0-3ad6aaad3540,ANLI,entailment
4,0,289606,WANLI,entailment
...,...,...,...,...
4245,15,296738,WANLI,entailment
4246,15,30618,WANLI,contradiction
4247,15,4688989316.jpg#3r1e,SNLI,entailment
4248,15,188121,WANLI,contradiction


In [1]:
import transformers

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
path = '/Users/ardsnijders/Documents/GitHub/Lisa/active_learning/resources/ard_data/multinli_1.0/multinli_1.0_train.jsonl'
# df = pd.read_json(path, lines=True)

In [51]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/Users/ardsnijders/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json
/Users/ardsnijders/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt


In [ ]:
df = pd.read_json(path)
df

In [59]:
def replace_labels(label):
    label_conversions = {'e': 'entailment',
                         'c': 'contradiction',
                         'n': 'neutral'}

    return label_conversions[label]

anli_dfs = []
for split in ['dev', 'test']:

    for data_round in ['R1', 'R2', 'R3']:

        data_path = '/Users/ardsnijders/Documents/GitHub/active_learning/resources/data/anli_v1.0/{}/{}.jsonl'.format(data_round, split)
        anli_dataset = pd.read_json(data_path, lines=True)
        anli_dataset = anli_dataset[['context', 'hypothesis', 'label', 'uid']]  # get rid of unnecessary columns
        anli_dataset['label'] = anli_dataset['label'].apply(replace_labels)  # ensures consistently named labels
        anli_dfs.append(anli_dataset)
        
    dataset = pd.concat(anli_dfs, axis=0)
    dataset.columns = ['Premise', 'Hypothesis', 'Label', 'ID']
    
    lengths = []
    for i, row in tqdm(dataset.iterrows()):
        premise = row['Premise']
        hypothesis = row['Hypothesis']

        # tokenize sentence and convert to sequence of ids
        tokenized_input_seq_pair = tokenizer.encode_plus(text=premise,
                                                              text_pair=hypothesis,
                                                              add_special_tokens=True,
                                                              max_length=2000,
                                                              padding='do_not_pad',
                                                              return_attention_mask=True,
                                                              return_token_type_ids=True,
                                                              return_tensors='pt',
                                                              truncation=False)

        ex_length = tokenized_input_seq_pair.input_ids.squeeze().size()[0]
        lengths.append(ex_length)
    
    lengths = np.array(lengths)
    print(lengths.max())
        
        


3200it [00:06, 527.16it/s]
52it [00:00, 516.04it/s]

189


6400it [00:12, 527.94it/s]

216


In [ ]:
345